In [91]:
import pandas as pd
import tensorflow as tf

# Standard python helper libraries.
import os, sys, re, json, time
import itertools
import collections
from IPython.display import display

# NumPy and SciPy for matrix ops
import numpy as np
import scipy.sparse

# Pandas because pandas are awesome
import pandas as pd
# Set pandas floating point display
pd.set_option('float_format', lambda f: "{0:.04f}".format(f))

# NLTK for NLP utils
import nltk

# Helper libraries
import utils
#import vocabulary

# Plotly imports.
import plotly.offline as plotly
plotly.offline.init_notebook_mode()
import plotly.graph_objs as go
from pprint import pprint
import sqlalchemy
import sqlite3

%matplotlib inline

from matplotlib import pyplot as plt
import time
import datetime
import gensim
from gensim import corpora, models, similarities
#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from pprint import pprint

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


# Table names in db

In [92]:
def importdb(db):
    
    conn = sqlite3.connect(db)
    c = conn.cursor()
    c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in c.fetchall():
        print (table)




In [93]:
importdb("reviews.db")

('reviews',)
('products',)
('categories',)


# Create pandas DF for reviews
* Need to merge with category csv pulled from Amazon API

In [94]:
conn = sqlite3.connect("reviews.db")
c = conn.cursor()
c.execute("Select * from reviews;")
review_table_0 = pd.DataFrame(c.fetchall())
# Didn't know what the first column was

review_table_0.columns = ["rating_id","rating","title","date","review","ASIN"]


review_table_0["rating"] = review_table_0.rating.apply(lambda x: int(x))
review_table_0["date"] = review_table_0.date.apply(lambda x: datetime.datetime.strptime(x, "%B %d, %Y"))
category_df = pd.read_csv("category_key.csv")
review_table = review_table_0.merge(category_df,on="ASIN",how="left")

review_table.head()

,rating_id,rating,title,date,review,ASIN,Unnamed: 0,Title,ProductGroup,ProductTypeName
0,RL17959WYMDQO,5,for sizing,2015-07-23,I wear a size 8 in women's so I ordered a size...,B06XRW26VW,0,"adidas Originals Baby Superstar I, White/Black...",Shoes,SHOES
1,RPDE4MBK5YZ02,4,Seem to run a little narrow,2017-11-19,"Own several pair of these, this pair seems to ...",B06XRW26VW,0,"adidas Originals Baby Superstar I, White/Black...",Shoes,SHOES
2,R1T62C0ORV3J9E,5,My daughter is in love with these,2017-01-12,My daughter is in love with these. the first p...,B06XRW26VW,0,"adidas Originals Baby Superstar I, White/Black...",Shoes,SHOES
3,R1OA8AZV9I45U8,5,Just love these shoes,2017-09-18,Just love these shoes! And cheaper than anywhe...,B06XRW26VW,0,"adidas Originals Baby Superstar I, White/Black...",Shoes,SHOES
4,R21370ICYS03G6,5,"but like always, Amazon came through",2016-09-07,Met my expectations perfectly! They were sold ...,B06XRW26VW,0,"adidas Originals Baby Superstar I, White/Black...",Shoes,SHOES


# A bit of exploration into dataset
* Also going to look as to where I want to subset dataset

In [95]:
review_table.shape

(62156, 10)

In [96]:
review_table.ProductTypeName.value_counts()

SHOES                   32158
SPORTING_GOODS          22425
SOCKSHOSIERY             3228
BAG                      1452
SHIRT                     970
PANTS                     862
SHORTS                    608
SWEATER                   301
OUTERWEAR                 141
TECHNICAL_SPORT_SHOE        6
TOYS_AND_GAMES              4
UNDERWEAR                   1
Name: ProductTypeName, dtype: int64

In [97]:
review_table.rating[review_table.ProductTypeName=="SHOES"].value_counts()

5    22098
4     5456
3     2080
1     1440
2     1084
Name: rating, dtype: int64

In [98]:
len(np.unique(review_table.ASIN[(review_table.rating<3) & (review_table.ProductTypeName=="SHOES")]))

224

# doc_complete is what I use as the raw corpus that will be trained on


In [99]:


doc_complete = [i for i in review_table.review[(review_table.rating<3) \
                & (review_table.ProductTypeName=="SHOES")]]

In [100]:
len(doc_complete)

2524

# Baseline results

In [101]:
n_topics = int(input("How many topics? "))

How many topics? 4


In [102]:
baseline_docs = [doc.split() for doc in doc_complete]

In [103]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(baseline_docs)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
#row is document, first value is word, second is wordcount
doc_term_matrix = [dictionary.doc2bow(doc) for doc in baseline_docs]

In [104]:
# Creating the object for LDA model using gensim library
Lda = models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=n_topics, id2word = dictionary, passes=50)

In [105]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)

/Users/Kevin/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



PreparedData(topic_coordinates=         Freq  cluster  topics       x       y
topic                                         
2     61.7020        1       1  0.2010 -0.0263
3     32.5959        1       2  0.1689  0.0513
1      3.3216        1       3 -0.1510 -0.1303
0      2.3805        1       4 -0.2189  0.1053, topic_info=     Category      Freq       Term     Total  loglift  logprob
term                                                          
108   Default  916.0000         is  916.0000  30.0000  30.0000
72    Default 3462.0000        the 3462.0000  29.0000  29.0000
1     Default  651.0000       size  651.0000  28.0000  28.0000
63    Default  450.0000        too  450.0000  27.0000  27.0000
33    Default 1288.0000         of 1288.0000  26.0000  26.0000
23    Default  368.0000         as  368.0000  25.0000  25.0000
142   Default  153.0000        big  153.0000  24.0000  24.0000
0     Default  600.0000        The  600.0000  23.0000  23.0000
134   Default  222.0000         me  222.0000  22.0000  22.0000
26    Default  142.0000          i  142.0000  21.0000  21.0000
141   Default  347.0000       very  347.0000  20.0000  20.0000
424   Default   87.0000      small   87.0000  19.0000  19.0000
180   Default  717.0000         on  717.0000  18.0000  18.0000
174   Default   84.0000      large   84.0000  17.0000  17.0000
154   Default  129.0000       Very  129.0000  16.0000  16.0000
49    Default   59.0000        Too   59.0000  15.0000  15.0000
314   Default  191.0000       than  191.0000  14.0000  14.0000
289   Default  123.0000        Not  123.0000  13.0000  13.0000
11    Default 1880.0000          a 1880.0000  12.0000  12.0000
223   Default  121.0000          2  121.0000  11.0000  11.0000
30    Default  236.0000      after  236.0000  10.0000  10.0000
270   Default  111.0000       This  111.0000   9.0000   9.0000
129   Default  461.0000       this  461.0000   8.0000   8.0000
633   Default   98.0000       what   98.0000   7.0000   7.0000
87    Default 2802.0000          I 2802.0000   6.0000   6.0000
192   Default   45.0000      sizes   45.0000   5.0000   5.0000
767   Default   81.0000     narrow   81.0000   4.0000   4.0000
178   Default   90.0000       much   90.0000   3.0000   3.0000
555   Default  146.0000        buy  146.0000   2.0000   2.0000
46    Default 2067.0000        and 2067.0000   1.0000   1.0000
...       ...       ...        ...       ...      ...      ...
8271   Topic4    2.5171     edges.    3.1339   3.5187  -6.7109
8850   Topic4    2.5146   patience    3.1360   3.5170  -6.7119
9198   Topic4    2.5145  HyperFlex    3.1370   3.5167  -6.7119
9214   Topic4    2.5145    reached    3.1370   3.5167  -6.7119
9225   Topic4    2.5145     Sonny.    3.1370   3.5167  -6.7119
9226   Topic4    2.5145      Marie    3.1370   3.5167  -6.7119
9227   Topic4    2.5145      Sonny    3.1370   3.5167  -6.7119
527    Topic4    2.3888       DONT    3.0147   3.5051  -6.7632
1016   Topic4    6.7485       cant    8.5420   3.5022  -5.7247
4411   Topic4    2.3835         im    3.0191   3.5015  -6.7654
713    Topic4    6.9511        Fit    9.8867   3.3856  -5.6951
6485   Topic4    3.8553       Made    5.3377   3.4125  -6.2845
124    Topic4    7.4334         No   15.8152   2.9829  -5.6280
1063   Topic4    4.9346       THIS    8.9884   3.1382  -6.0377
582    Topic4    8.4020       fake   29.5568   2.4800  -5.5055
62     Topic4    7.9504        Way   27.6286   2.4922  -5.5608
1458   Topic4    4.0529       Fake    7.7243   3.0929  -6.2345
1674   Topic4    4.2879      them!    8.7913   3.0199  -6.1782
2358   Topic4    3.8852         IS    7.3532   3.0999  -6.2768
1885   Topic4    3.9331         DO    7.6101   3.0778  -6.2645
1082   Topic4    8.2122        NOT   34.3550   2.3067  -5.5284
1217   Topic4    3.4278        AND    5.8762   3.1988  -6.4021
26     Topic4   13.0758          i  142.5124   1.3492  -5.0632
134    Topic4   11.8392         me  222.9310   0.8024  -5.1626
900    Topic4    5.1752          A   29.7726   1.9881  -5.9901
390    Topic4    3.41

# Cleanse data
* Remove stop words (we see a lot of examples of the, it, etc.) 
    * Also will remove numbers as they have shown little help
    * Remove words that only show up once as well, as they will do little since LDA is looking for like topics using words as features
* Lemma of words (use available package to get lemma of words since they will have similar meaning)
* Remove punctuation as that creates unique features when they should not be unique


In [106]:
# Finding all words that only appear once in corpus
all_words = []

for doc in doc_complete:
    all_words.extend(doc.split())
from collections import Counter
counts = Counter(all_words)   

wc = pd.DataFrame.from_dict(counts,orient="index")

one_time = set(wc.index[wc[0]==1])

In [107]:
from nltk.corpus import stopwords
import re
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

p_stemmer = PorterStemmer()

# Removing every day words that provide little use in classification purposes
stop = set(stopwords.words('english'))
#Extra stop words below
stop.update({'sneaker','adidas','shoe','shoes','boost','wear','feet','foot','pair','sandal','sandals','pairs'\
             'basketball','like','likes','dont','one','bought','buy','bad','return','returned','returns'}) 
stop.update(one_time)
exclude = set(string.punctuation) 

def clean(doc):
    numbers = re.sub(r'\d\d*','',doc)
    comfort = re.sub(r'not comfortable','uncomfort',numbers)
    punc_free = ''.join(ch for ch in " ".join(i for i in comfort.lower().split()) if ch not in exclude)
    stop_free = " ".join([i for i in punc_free.split() if i not in stop])
    normalized = " ".join(p_stemmer.stem(word) for word in stop_free.split())
    
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete] 

In [108]:
# Looking through doc_clean for any more potential stop words I may have missed
all_words_clean = []

for doc in doc_clean:
    all_words_clean.extend(doc)

counts = Counter(all_words_clean)   

clean_wc = pd.DataFrame.from_dict(counts,orient="index")
clean_wc.sort_values(by=0,ascending=False,inplace=True)
clean_wc.head(15)

,0
size,907
fit,416
look,405
order,326
would,274
use,262
big,261
small,259
month,255
back,234


In [109]:
bigram = gensim.models.Phrases(baseline_docs)

In [110]:
from_base = [bigram[line] for line in baseline_docs]

/Users/Kevin/anaconda/lib/python3.6/site-packages/gensim/models/phrases.py:316: UserWarning:

For a faster implementation, use the gensim.models.phrases.Phraser class



In [111]:
from_base[1]

['Junk.',
 'These_are',
 'not',
 'name',
 'brand',
 'Adidas',
 'as',
 'description',
 'depicts.',
 'The',
 'i',
 'nsole',
 'comes',
 'ouit',
 'after',
 'one',
 'day',
 'of',
 'wear.',
 'And',
 'they_are',
 'cheap',
 'looking.',
 'Dnt',
 'bother',
 'purchasing']

In [112]:
# Chose to run this after cleansing due to the results that came up if done before cleansing as seen above

doc_clean = [bigram[line] for line in doc_clean]

/Users/Kevin/anaconda/lib/python3.6/site-packages/gensim/models/phrases.py:316: UserWarning:

For a faster implementation, use the gensim.models.phrases.Phraser class



In [113]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary_clean = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
#row is document, first value is word, second is wordcount
doc_term_matrix_clean = [dictionary_clean.doc2bow(doc) for doc in doc_clean]

In [130]:
gensim.models.ldamodel.LdaModel?

In [146]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel_clean = Lda(doc_term_matrix_clean, num_topics=n_topics, id2word = dictionary_clean,passes=100,iterations=200,random_state = 1) #,alpha=np.asarray([10]*(n_topics)))

In [147]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel_clean, doc_term_matrix_clean, dictionary_clean)

/Users/Kevin/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



PreparedData(topic_coordinates=         Freq  cluster  topics       x       y
topic                                         
2     32.3018        1       1 -0.1424  0.1077
1     29.1561        1       2 -0.0536 -0.1981
3     19.7103        1       3  0.2349  0.0300
0     18.8318        1       4 -0.0388  0.0604, topic_info=     Category     Freq        Term    Total  loglift  logprob
term                                                         
0     Default 718.0000        size 718.0000  30.0000  30.0000
29    Default 289.0000         big 289.0000  29.0000  29.0000
26    Default 288.0000       order 288.0000  28.0000  28.0000
62    Default 181.0000       small 181.0000  27.0000  27.0000
147   Default 206.0000       month 206.0000  26.0000  26.0000
78    Default 149.0000        larg 149.0000  25.0000  25.0000
28    Default 191.0000         way 191.0000  24.0000  24.0000
445   Default 166.0000        week 166.0000  23.0000  23.0000
2     Default 394.0000         fit 394.0000  22.0000  22.0000
21    Default 103.0000         box 103.0000  21.0000  21.0000
104   Default 223.0000        sole 223.0000  20.0000  20.0000
205   Default 132.0000        walk 132.0000  19.0000  19.0000
414   Default 131.0000        hurt 131.0000  18.0000  18.0000
137   Default 192.0000         run 192.0000  17.0000  17.0000
285   Default 122.0000   uncomfort 122.0000  16.0000  16.0000
374   Default 173.0000     qualiti 173.0000  15.0000  15.0000
54    Default 106.0000        fake 106.0000  14.0000  14.0000
152   Default  77.0000       women  77.0000  13.0000  13.0000
510   Default 104.0000      rubber 104.0000  12.0000  12.0000
145   Default  66.0000         men  66.0000  11.0000  11.0000
74    Default 226.0000         use 226.0000  10.0000  10.0000
441   Default 109.0000        last 109.0000   9.0000   9.0000
370   Default 134.0000      narrow 134.0000   8.0000   8.0000
474   Default  88.0000        play  88.0000   7.0000   7.0000
411   Default  80.0000        pain  80.0000   6.0000   6.0000
76    Default  55.0000       white  55.0000   5.0000   5.0000
415   Default 169.0000     comfort 169.0000   4.0000   4.0000
434   Default  84.0000      expect  84.0000   3.0000   3.0000
505   Default  71.0000     support  71.0000   2.0000   2.0000
391   Default  83.0000        sock  83.0000   1.0000   1.0000
...       ...      ...         ...      ...      ...      ...
1037   Topic4  21.5289       notic  27.4140   1.4280  -5.6867
158    Topic4  50.3297      origin  78.7108   1.2224  -4.8375
35     Topic4  62.9278      receiv 105.4025   1.1538  -4.6141
109    Topic4  40.1101        real  61.1815   1.2474  -5.0645
839    Topic4  19.7657       stain  25.3070   1.4225  -5.7722
659    Topic4  25.6232       weird  35.7847   1.3356  -5.5126
87     Topic4  53.7711        came  96.7024   1.0827  -4.7714
41     Topic4  40.8879       anoth  67.4093   1.1697  -5.0453
447    Topic4  26.7285         ill  37.9648   1.3187  -5.4704
153    Topic4  58.3932         got 121.0646   0.9405  -4.6889
275    Topic4  60.2788        want 133.5836   0.8739  -4.6572
149    Topic4  38.1818        side  67.8399   1.0948  -5.1138
259    Topic4  51.1263      differ 110.2637   0.9010  -4.8218
110    Topic4  65.3401     product 163.9197   0.7499  -4.5765
297    Topic4  58.6657        made 142.1353   0.7847  -4.6843
16     Topic4  96.0320        look 342.0459   0.3994  -4.1915
353    Topic4  32.7150      pictur  55.7803   1.1360  -5.2683
144    Topic4  39.6702         say  86.5445   0.8896  -5.0755
79     Topic4  38.8426        cant  83.3535   0.9061  -5.0966
8      Topic4  57.9678        time 198.8434   0.4370  -4.6962
38     Topic4  67.2759        back 290.6947   0.2062  -4.5473
13     Topic4  40.8622        come  98.5122   0.7896  -5.0459
100    Topic4  54.2885         get 188.3526   0.4256  -4.7618
73     Topic4  37.2053     sneaker  80.7474   0.8948  -5.1397
215    Topic4  38.6335       color  88.1923   0.8442  -5.1020
151    Topic4  47.5301      realli 165.8623   0.4198  -4.8948
26     Topic4  56

# POS tagging

In [148]:
#baseline_docs = [doc.split() for doc in doc_complete]

POS = pd.DataFrame(nltk.pos_tag(all_words))
POS.columns = ["word","POS"]

In [149]:
adjectives = np.unique(POS.word[POS.POS.str.contains("JJ")])

In [150]:
from nltk.corpus import stopwords
import re
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

p_stemmer = PorterStemmer()
import string
# Removing every day words that provide little use in classification purposes
#stop = set(stopwords.words('english'))
#Extra stop words below
stop.update({'sneaker','adidas','shoe','shoes','boost','0','feet','foot','pair','sandal','sandals','pairs'\
             'basketball','like','likes','dont','one','bought','buy','bad'})
#stop.update(one_time)
#stop.update(extra_removal_2)
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean_only_adj(doc):
    adjectivized = " ".join([word for word in doc.split() if word in adjectives])
    numbers = re.sub(r'\d\d*','',adjectivized)
    punc_free = ''.join(ch for ch in " ".join(i for i in numbers.lower().split()) if ch not in exclude)
    stop_free = " ".join([i for i in punc_free.split() if i not in stop])
    #punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
   # normalized = " ".join(lemma.lemmatize(word) for word in stop_free.split())
    normalized = " ".join(p_stemmer.stem(word) for word in stop_free.split())
    
    
    return normalized

doc_adj = [clean_only_adj(doc).split() for doc in doc_complete] 





In [152]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary_adj = corpora.Dictionary(doc_adj)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
#row is document, first value is word, second is wordcount
doc_term_matrix_adj = [dictionary_adj.doc2bow(doc) for doc in doc_adj]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel_adj = Lda(doc_term_matrix_adj, num_topics=n_topics, id2word = dictionary_adj, passes=100,random_state=1,iterations=200 )

In [153]:
pyLDAvis.gensim.prepare(ldamodel_adj, doc_term_matrix_adj, dictionary_adj)

/Users/Kevin/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



PreparedData(topic_coordinates=         Freq  cluster  topics       x       y
topic                                         
0     32.5592        1       1  0.0807  0.0342
2     25.2560        1       2  0.2368  0.0665
3     23.8660        1       3 -0.0710 -0.2467
1     18.3189        1       4 -0.2465  0.1461, topic_info=     Category     Freq        Term    Total  loglift  logprob
term                                                         
10    Default 303.0000         big 303.0000  30.0000  30.0000
23    Default 198.0000       small 198.0000  29.0000  29.0000
21    Default 173.0000        size 173.0000  28.0000  28.0000
162   Default 150.0000     qualiti 150.0000  27.0000  27.0000
31    Default 165.0000        larg 165.0000  26.0000  26.0000
41    Default 124.0000        sole 124.0000  25.0000  25.0000
159   Default 128.0000      narrow 128.0000  24.0000  24.0000
1     Default 322.0000         fit 322.0000  23.0000  23.0000
26    Default 174.0000       order 174.0000  22.0000  22.0000
173   Default 103.0000       tight 103.0000  21.0000  21.0000
39    Default  82.0000     smaller  82.0000  20.0000  20.0000
40    Default  97.0000         box  97.0000  19.0000  19.0000
13    Default 274.0000        back 274.0000  18.0000  18.0000
108   Default  90.0000      differ  90.0000  17.0000  17.0000
217   Default  94.0000      rubber  94.0000  16.0000  16.0000
181   Default 154.0000     comfort 154.0000  15.0000  15.0000
28    Default 120.0000         use 120.0000  14.0000  14.0000
20    Default 104.0000         son 104.0000  13.0000  13.0000
90    Default  95.0000        half  95.0000  12.0000  12.0000
29    Default  63.0000       white  63.0000  11.0000  11.0000
7     Default 252.0000        look 252.0000  10.0000  10.0000
265   Default  70.0000        high  70.0000   9.0000   9.0000
594   Default  69.0000        walk  69.0000   8.0000   8.0000
57    Default  53.0000        less  53.0000   7.0000   7.0000
105   Default  50.0000       women  50.0000   6.0000   6.0000
106   Default  49.0000     leather  49.0000   5.0000   5.0000
161   Default  51.0000        poor  51.0000   4.0000   4.0000
74    Default  48.0000        true  48.0000   3.0000   3.0000
44    Default  55.0000        real  55.0000   2.0000   2.0000
6     Default  49.0000       cheap  49.0000   1.0000   1.0000
...       ...      ...         ...      ...      ...      ...
544    Topic4   5.2645     ridicul   6.0043   1.5658  -6.1275
521    Topic4   4.7396       typic   5.4560   1.5565  -6.2326
167    Topic4   4.7219          us   5.4562   1.5527  -6.2363
386    Topic4   4.0610       freak   4.7715   1.5360  -6.3871
801    Topic4   4.1658       giant   4.9084   1.5332  -6.3616
159    Topic4 106.4556      narrow 128.9213   1.5058  -3.1208
21     Topic4 141.4662        size 173.7226   1.4918  -2.8365
271    Topic4  25.5135        heal  30.9385   1.5044  -4.5493
144    Topic4  41.9624       wrong  53.9561   1.4458  -4.0518
90     Topic4  64.9247        half  95.9960   1.3062  -3.6153
20     Topic4  63.2260         son 104.0428   1.1992  -3.6418
26     Topic4  94.0937       order 174.8175   1.0778  -3.2442
422    Topic4  17.3319       whole  24.9606   1.3325  -4.9360
120    Topic4  12.9236      refund  18.6031   1.3330  -5.2295
165    Topic4  15.7897         red  26.4104   1.1828  -5.0292
86     Topic4  21.5683        wish  42.1689   1.0268  -4.7173
119    Topic4  37.6202   uncomfort 107.0864   0.6511  -4.1610
1      Topic4  71.9828         fit 322.1660   0.1986  -3.5121
99     Topic4  18.4607        full  36.2576   1.0222  -4.8729
381    Topic4  20.2248       least  43.1004   0.9406  -4.7816
93     Topic4  24.7896        sent  61.9820   0.7808  -4.5781
13     Topic4  55.7957        back 274.2004   0.1051  -3.7668
22     Topic4  34.8721  disappoint 170.6361   0.1094  -4.2368
88     Topic4  27.5842       great 136.3164   0.0995  -4.4713
33     Topic4  19.4617        much 109.3414  -0.0288  -4.8201
61     Topic4  20.7338        good 166.9408  -0.3886  -4.7568
157    Topic4  12

In [67]:
def table_it(corpus,model):
    topic_assignment = dict()
    for doc,all_topics in enumerate(model.get_document_topics(corpus)):
        maxim = 0
        final_doc = None
        final_top = None
        for topic in all_topics:        
            if topic[1]>maxim:
                final_doc,final_top, maxim = doc,topic[0],topic[1]
        topic_assignment[doc] = [final_top,maxim,doc_complete[doc]]


    topic_df_0 = pd.DataFrame.from_dict(topic_assignment,orient="index")
    topic_df_0.columns = ["Topic","prob","text"]

    topic_list = []
    for topic in range(n_topics):
        x=[]
        for i in model.show_topic(topicid=topic):
            x.append(i[0])

        topic_list.append([topic, str(x)])

    topic_names= pd.DataFrame(topic_list)
    topic_names.columns = ["Topic","Topic_header"]

    topic_df = topic_df_0.merge(topic_names,on="Topic")
    return topic_df

#topic_assignment

In [68]:
adj_table = table_it(doc_term_matrix_adj,ldamodel_adj)
#idf_table = table_it(corpus_tfidf,ldamodel_clean_tfidf)
clean_table = table_it(doc_term_matrix_clean,ldamodel_clean)
baseline_table = table_it(doc_term_matrix,ldamodel)

In [69]:
clean_table[clean_table.Topic==1].sort_values(by="prob",ascending=False).head(10)

,Topic,prob,text,Topic_header
2107,1,0.7974,"I play a lot of golf, and I hate wet feet. I h...","['size', 'order', 'big', 'back', 'fit', 'recei..."
1994,1,0.6624,After reading reviews I ordered a 1/2 size dow...,"['size', 'order', 'big', 'back', 'fit', 'recei..."
2110,1,0.5458,These shoes are awesome! $200 at the retail ri...,"['size', 'order', 'big', 'back', 'fit', 'recei..."
2060,1,0.5094,I received these shoes as a gift. I loved them...,"['size', 'order', 'big', 'back', 'fit', 'recei..."
1544,1,0.4947,I literally JUST placed my order a couple of h...,"['size', 'order', 'big', 'back', 'fit', 'recei..."
2027,1,0.4774,This was a serious disappointment. I read all ...,"['size', 'order', 'big', 'back', 'fit', 'recei..."
1646,1,0.4748,"I love the style of the shoes, but I am not ha...","['size', 'order', 'big', 'back', 'fit', 'recei..."
1945,1,0.4534,I made the mistake of ordering these without r...,"['size', 'order', 'big', 'back', 'fit', 'recei..."
2080,1,0.4472,Ordered my regular shoe size even though some ...,"['size', 'order', 'big', 'back', 'fit', 'recei..."
2100,1,0.4468,The picture depicts All white shoe with off wh...,"['size', 'order', 'big', 'back', 'fit', 'recei..."


In [71]:
clean_table.text.loc[1994]

'After reading reviews I ordered a 1/2 size down from the usual Adidas I have. (Interesting that I have same style Adidas in different patterns and I have to order a different size. They fit almost as expected. The rubber shell tip on the right foot dips at the seam of the shoe and rubber tip. It is confining but I am hoping I can work it to the correct position. Maybe stuffing the shoe will work? Regardless, they are the correct length with sizing a 1/2 size smaller, and less expensive and in my size as opposed to the official Adidas website. The Rubber, however is off-white, not white like the leather which is kinda odd because my other Adidas Original Superstars have white rubber. This gives them a weird "aged rubber" kind of appearance of which I\'m not particularly fond. They are definitely not my favorite pair of Adidas.\nUPDATE: After wearing these shoes a little the soles showed that under that off-white rubber was YELLOW RUBBER! My other Original (From the Adidas website) do n

In [137]:
#ldamodel_clean.get_document_topics([clean_table.text.loc[2498].split()])
[i for i in ldamodel_clean.get_document_topics(doc_term_matrix_clean)[:1]]

[[(0, 0.92283169052425418),
  (1, 0.025381246502742837),
  (2, 0.02593131089848456),
  (3, 0.025855752074518519)]]

# End of notebook

In [353]:
from autocorrect import spell

In [357]:
baseline_docs[1]

['Junk.',
 'These',
 'are',
 'not',
 'name',
 'brand',
 'Adidas',
 'as',
 'description',
 'depicts.',
 'The',
 'i',
 'nsole',
 'comes',
 'ouit',
 'after',
 'one',
 'day',
 'of',
 'wear.',
 'And',
 'they',
 'are',
 'cheap',
 'looking.',
 'Dnt',
 'bother',
 'purchasing']

In [360]:
[i for i in baseline_docs[1]]

['Junk.',
 'These',
 'are',
 'not',
 'name',
 'brand',
 'Adidas',
 'as',
 'description',
 'depicts.',
 'The',
 'i',
 'nsole',
 'comes',
 'ouit',
 'after',
 'one',
 'day',
 'of',
 'wear.',
 'And',
 'they',
 'are',
 'cheap',
 'looking.',
 'Dnt',
 'bother',
 'purchasing']

# N - grams...need different means of cleansing data

In [ ]:
input_list = ['all', 'this', 'happened', 'more', 'or', 'less']

def find_bigrams(input_list):
  bigram_list = []
  for i in range(len(input_list)-1):
      bigram_list.append((input_list[i], input_list[i+1]))
  return bigram_list

In [195]:
def clean_gram(doc):
    numbers = re.sub(r'\d\d*','',doc)
    punc_free = ''.join(ch for ch in " ".join(i for i in numbers.lower().split()) if ch not in exclude)
    #stop_free = " ".join([i for i in punc_free.split() if i not in stop])
    #punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    
    return normalized

doc_clean_gram = [clean_gram(doc).split() for doc in doc_complete] 

In [196]:
# N-gram set
n_gram_clean = []

for doc in doc_clean_gram:
    #added in beginning and end of review
    start = ["<s>"]
    start.extend(doc)
    start.extend(["<e>"])
    doc_set = []
    for i,word in enumerate(start):
        if i > 0:
            n_grams = [start[i-1],start[i]]
            #print(n_grams)
            doc_set.append(str(n_grams))
    n_gram_clean.append(doc_set)
        
        


In [197]:
all_words = []

for doc in n_gram_clean:
    all_words.extend(doc)
from collections import Counter
counts = Counter(all_words)   

wc = pd.DataFrame.from_dict(counts,orient="index")

one_time = set(wc.index[wc[0]==1])

In [198]:
wc[wc[0]>1].shape

(29394, 1)

In [199]:
#''.join(ch for ch in " ".join(i for i in doc_complete[0].lower().split()) if ch not in ["'","."])

In [200]:
#Testing removal of stop words

for i in doc_clean:
    if 'adidas' in i:
        print (i)

In [201]:
import gensim
from gensim import corpora, models, similarities

In [202]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(n_gram_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
#row is document, first value is word, second is wordcount
doc_term_matrix = [dictionary.doc2bow(doc) for doc in n_gram_clean]

2017-12-09 15:14:18,704 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-12-09 15:14:19,038 : INFO : adding document #10000 to Dictionary(44313 unique tokens: ["['<s>', 'i']", "['i', 'wear']", "['wear', 'a']", "['a', 'size']", "['size', 'in']"]...)
2017-12-09 15:14:19,429 : INFO : adding document #20000 to Dictionary(82818 unique tokens: ["['<s>', 'i']", "['i', 'wear']", "['wear', 'a']", "['a', 'size']", "['size', 'in']"]...)
2017-12-09 15:14:19,532 : INFO : built Dictionary(90327 unique tokens: ["['<s>', 'i']", "['i', 'wear']", "['wear', 'a']", "['a', 'size']", "['size', 'in']"]...) from 22098 documents (total 428392 corpus positions)


# Exploring tfidf

In [203]:
tfidf = models.TfidfModel(doc_term_matrix)

2017-12-09 15:14:23,633 : INFO : collecting document frequencies
2017-12-09 15:14:23,634 : INFO : PROGRESS: processing document #0
2017-12-09 15:14:23,685 : INFO : PROGRESS: processing document #10000
2017-12-09 15:14:23,755 : INFO : PROGRESS: processing document #20000
2017-12-09 15:14:23,779 : INFO : calculating IDF weights for 22098 documents and 90326 features (420912 matrix non-zeros)


In [204]:
corpus_tfidf = tfidf[doc_term_matrix]

#for doc in corpus_tfidf[:5]:
#    print (doc)

# Look at the difference 
print("original: \n", doc_term_matrix[0],"\n\ntfidf: \n",corpus_tfidf[0])

original: 
 [(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1)] 

tfidf: 
 [(0, 0.04790435702358642), (1, 0.08006875162225356), (2, 0.08272119595072334), (3, 0.1420636334122339), (4, 0.10487252983127521), (5, 0.10919343929048948), (6, 0.14783689846197953), (7, 0.08124558872921736), (8, 0.07998711861658718), (9, 0.08927911424628349), (10, 0.16470028376147533), (11, 0.12785405924860163), (12, 0.07392608487052689), (13, 0.1608008228455421), (14, 0.19045063921517114), (15, 0.21394756276473625), (16, 0.19045063921517114), (17, 0.21394756276473625), (18, 0.21394756276473625), (19, 0.19045063921517114), (20, 0.12401834747530399), (21, 0.179525191

In [205]:
# Just looking at format of dictionary...can ignore
for i in dictionary:
    if i < 10:
        print (i,dictionary[i])
    else:
        break
    

0 ['<s>', 'i']
1 ['i', 'wear']
2 ['wear', 'a']
3 ['a', 'size']
4 ['size', 'in']
5 ['in', 'woman']
6 ['woman', 'so']
7 ['so', 'i']
8 ['i', 'ordered']
9 ['ordered', 'a']


# ACTUAL MODEL

In [206]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(corpus_tfidf, num_topics=n_topics, id2word = dictionary, passes=100)

2017-12-09 15:14:27,565 : INFO : using symmetric alpha at 0.1
2017-12-09 15:14:27,566 : INFO : using symmetric eta at 1.1070886888748657e-05
2017-12-09 15:14:27,581 : INFO : using serial LDA version on this node
2017-12-09 15:14:33,414 : INFO : running online (multi-pass) LDA training, 10 topics, 100 passes over the supplied corpus of 22098 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2017-12-09 15:14:33,476 : INFO : PROGRESS: pass 0, at document #2000/22098
2017-12-09 15:14:34,849 : INFO : merging changes from 2000 documents into a model of 22098 documents
2017-12-09 15:14:35,103 : INFO : topic #6 (0.100): 0.003*"['my', 'daughter']" + 0.003*"['daughter', 'love']" + 0.003*"['love', 'them']" + 0.003*"['so', 'cute']" + 0.003*"['cute', '<e>']" + 0.002*"['<s>', 'so']" + 0.002*"['<s>', 'great']" + 0.002*"['<s>', 'i']" + 0.002*"['great', '<e>']" + 0.002*"['basketball', '<e>']"
2017-12-

2017-12-09 15:14:42,952 : INFO : topic #2 (0.100): 0.002*"['stan', 'smith']" + 0.002*"['for', 'my']" + 0.002*"['<s>', 'my']" + 0.002*"['my', 'son']" + 0.002*"['year', 'old']" + 0.002*"['my', 'year']" + 0.002*"['well', '<e>']" + 0.002*"['excellent', 'product']" + 0.002*"['love', 'his']" + 0.002*"['<s>', 'excellent']"
2017-12-09 15:14:42,954 : INFO : topic #1 (0.100): 0.008*"['<s>', 'my']" + 0.007*"['son', 'love']" + 0.007*"['them', '<e>']" + 0.006*"['love', 'them']" + 0.006*"['my', 'son']" + 0.004*"['daughter', 'love']" + 0.004*"['my', 'daughter']" + 0.003*"['love', 'these']" + 0.003*"['<s>', 'cool']" + 0.003*"['these', '<e>']"
2017-12-09 15:14:42,957 : INFO : topic #5 (0.100): 0.017*"['shoe', '<e>']" + 0.016*"['great', 'shoe']" + 0.013*"['<s>', 'great']" + 0.012*"['<s>', 'love']" + 0.009*"['them', '<e>']" + 0.009*"['love', 'them']" + 0.004*"['love', 'these']" + 0.004*"['these', 'shoe']" + 0.003*"['quality', '<e>']" + 0.003*"['excellent', 'shoe']"
2017-12-09 15:14:42,959 : INFO : topic 

2017-12-09 15:14:50,700 : INFO : topic #0 (0.100): 0.007*"['<s>', 'awesome']" + 0.005*"['awesome', '<e>']" + 0.003*"['awesome', 'shoe']" + 0.003*"['thanks', '<e>']" + 0.002*"['ok', '<e>']" + 0.002*"['<s>', 'ok']" + 0.002*"['<s>', 'thanks']" + 0.002*"['and', 'stylish']" + 0.002*"['<s>', 'loved']" + 0.002*"['loved', 'them']"
2017-12-09 15:14:50,704 : INFO : topic diff=0.436402, rho=0.333333
2017-12-09 15:14:53,901 : INFO : -14.381 per-word bound, 21335.9 perplexity estimate based on a held-out corpus of 2000 documents with 6956 words
2017-12-09 15:14:53,902 : INFO : PROGRESS: pass 0, at document #20000/22098
2017-12-09 15:14:55,174 : INFO : merging changes from 2000 documents into a model of 22098 documents
2017-12-09 15:14:55,441 : INFO : topic #3 (0.100): 0.009*"['comfortable', '<e>']" + 0.007*"['very', 'comfortable']" + 0.006*"['<s>', 'very']" + 0.003*"['so', 'comfortable']" + 0.002*"['my', 'husband']" + 0.002*"['extremely', 'comfortable']" + 0.002*"['<s>', 'so']" + 0.002*"['cute', '<

2017-12-09 15:15:00,167 : INFO : merging changes from 2000 documents into a model of 22098 documents
2017-12-09 15:15:00,425 : INFO : topic #8 (0.100): 0.009*"['excellent', '<e>']" + 0.008*"['<s>', 'excellent']" + 0.003*"['classic', '<e>']" + 0.003*"['<s>', '<e>']" + 0.001*"['down', 'a']" + 0.001*"['an', 'and']" + 0.001*"['my', 'usual']" + 0.001*"['usual', 'size']" + 0.001*"['full', 'size']" + 0.001*"['a', 'full']"
2017-12-09 15:15:00,427 : INFO : topic #2 (0.100): 0.003*"['a', 'size']" + 0.003*"['size', 'down']" + 0.003*"['i', 'ordered']" + 0.003*"['they', 'are']" + 0.003*"['fit', 'perfectly']" + 0.003*"['<s>', 'i']" + 0.002*"['ordered', 'a']" + 0.002*"['the', 'shoe']" + 0.002*"['they', 'fit']" + 0.002*"['and', 'they']"
2017-12-09 15:15:00,428 : INFO : topic #3 (0.100): 0.005*"['comfortable', '<e>']" + 0.004*"['<s>', 'very']" + 0.004*"['very', 'comfortable']" + 0.002*"['cool', '<e>']" + 0.002*"['a', 'woman']" + 0.002*"['quality', 'comfortable']" + 0.002*"['is', 'perfect']" + 0.002*"['

2017-12-09 15:15:05,488 : INFO : topic #4 (0.100): 0.016*"['great', '<e>']" + 0.012*"['perfect', '<e>']" + 0.012*"['good', '<e>']" + 0.012*"['<s>', 'perfect']" + 0.012*"['<s>', 'great']" + 0.011*"['<s>', 'good']" + 0.009*"['it', '<e>']" + 0.009*"['love', 'it']" + 0.005*"['<s>', 'love']" + 0.004*"['fit', 'great']"
2017-12-09 15:15:05,489 : INFO : topic #3 (0.100): 0.004*"['comfortable', '<e>']" + 0.004*"['very', 'comfortable']" + 0.004*"['<s>', 'very']" + 0.002*"['my', 'husband']" + 0.002*"['cool', '<e>']" + 0.001*"['husband', 'love']" + 0.001*"['fit', 'just']" + 0.001*"['nice', 'fit']" + 0.001*"['right', '<e>']" + 0.001*"['just', 'a']"
2017-12-09 15:15:05,493 : INFO : topic diff=0.323854, rho=0.276829
2017-12-09 15:15:05,567 : INFO : PROGRESS: pass 1, at document #14000/22098
2017-12-09 15:15:06,470 : INFO : merging changes from 2000 documents into a model of 22098 documents
2017-12-09 15:15:06,746 : INFO : topic #9 (0.100): 0.007*"['nice', 'shoe']" + 0.007*"['very', 'good']" + 0.007*"

...

2017-12-09 15:39:44,742 : INFO : topic #9 (0.100): 0.009*"['very', 'good']" + 0.008*"['<s>', 'very']" + 0.008*"['nice', 'shoe']" + 0.006*"['a', 'expected']" + 0.006*"['expected', '<e>']" + 0.006*"['<s>', 'nice']" + 0.005*"['shoe', '<e>']" + 0.005*"['good', 'shoe']" + 0.005*"['good', '<e>']" + 0.004*"['<s>', 'good']"
2017-12-09 15:39:44,746 : INFO : topic diff=0.056423, rho=0.095325
2017-12-09 15:39:44,824 : INFO : PROGRESS: pass 98, at document #14000/22098
2017-12-09 15:39:45,571 : INFO : merging changes from 2000 documents into a model of 22098 documents
2017-12-09 15:39:45,834 : INFO : topic #8 (0.100): 0.008*"['excellent', '<e>']" + 0.008*"['<s>', 'excellent']" + 0.003*"['classic', '<e>']" + 0.003*"['<s>', '<e>']" + 0.002*"['favorite', 'shoe']" + 0.001*"['shoe', 'ever']" + 0.001*"['ever', '<e>']" + 0.001*"['<s>', 'best']" + 0.001*"['my', 'favorite']" + 0.001*"['your', 'foot']"
2017-12-09 15:39:45,836 : INFO : topic #6 (0.100): 0.005*"['product', '<e>']" + 0.004*"['great', 'product'

2017-12-09 15:39:52,663 : INFO : topic #8 (0.100): 0.007*"['excellent', '<e>']" + 0.007*"['<s>', 'excellent']" + 0.003*"['<s>', '<e>']" + 0.002*"['classic', '<e>']" + 0.002*"['your', 'foot']" + 0.002*"['favorite', 'shoe']" + 0.001*"['shoe', 'ever']" + 0.001*"['ever', '<e>']" + 0.001*"['<s>', 'best']" + 0.001*"['my', 'favorite']"
2017-12-09 15:39:52,665 : INFO : topic #5 (0.100): 0.016*"['<s>', 'love']" + 0.014*"['shoe', '<e>']" + 0.013*"['them', '<e>']" + 0.012*"['great', 'shoe']" + 0.012*"['love', 'them']" + 0.010*"['<s>', 'great']" + 0.004*"['love', 'these']" + 0.003*"['i', 'love']" + 0.003*"['these', 'shoe']" + 0.003*"['love', 'love']"
2017-12-09 15:39:52,667 : INFO : topic #6 (0.100): 0.005*"['product', '<e>']" + 0.004*"['great', 'product']" + 0.003*"['<s>', 'so']" + 0.002*"['<s>', 'great']" + 0.002*"['<s>', 'comfy']" + 0.001*"['👍', '<e>']" + 0.001*"['<s>', '👍']" + 0.001*"['very', 'pleased']" + 0.001*"['ive', 'ever']" + 0.001*"['so', 'far']"
2017-12-09 15:39:52,669 : INFO : topic #

2017-12-09 15:39:56,360 : INFO : topic #5 (0.100): 0.017*"['<s>', 'love']" + 0.014*"['them', '<e>']" + 0.013*"['love', 'them']" + 0.013*"['shoe', '<e>']" + 0.012*"['great', 'shoe']" + 0.010*"['<s>', 'great']" + 0.004*"['love', 'these']" + 0.004*"['excelente', '<e>']" + 0.003*"['<s>', 'excelente']" + 0.003*"['i', 'love']"
2017-12-09 15:39:56,362 : INFO : topic #2 (0.100): 0.004*"['they', 'are']" + 0.004*"['for', 'my']" + 0.004*"['<s>', 'i']" + 0.004*"['these', 'shoe']" + 0.003*"['my', 'son']" + 0.003*"['a', 'size']" + 0.003*"['fit', 'perfectly']" + 0.003*"['the', 'shoe']" + 0.003*"['and', 'they']" + 0.003*"['love', 'them']"
2017-12-09 15:39:56,366 : INFO : topic diff=0.041094, rho=0.094895
2017-12-09 15:39:56,442 : INFO : PROGRESS: pass 99, at document #8000/22098
2017-12-09 15:39:57,176 : INFO : merging changes from 2000 documents into a model of 22098 documents
2017-12-09 15:39:57,427 : INFO : topic #4 (0.100): 0.015*"['great', '<e>']" + 0.013*"['<s>', 'perfect']" + 0.013*"['perfect',

2017-12-09 15:40:01,811 : INFO : topic #6 (0.100): 0.005*"['product', '<e>']" + 0.004*"['great', 'product']" + 0.002*"['<s>', 'great']" + 0.002*"['<s>', 'so']" + 0.002*"['👍', '<e>']" + 0.002*"['<s>', '👍']" + 0.002*"['<s>', 'comfy']" + 0.001*"['very', 'pleased']" + 0.001*"['ive', 'ever']" + 0.001*"['so', 'far']"
2017-12-09 15:40:01,813 : INFO : topic #1 (0.100): 0.008*"['<s>', 'my']" + 0.006*"['son', 'love']" + 0.005*"['them', '<e>']" + 0.005*"['daughter', 'love']" + 0.005*"['my', 'son']" + 0.005*"['love', 'them']" + 0.004*"['my', 'daughter']" + 0.004*"['these', '<e>']" + 0.004*"['love', 'these']" + 0.003*"['<s>', 'excelent']"
2017-12-09 15:40:01,815 : INFO : topic #8 (0.100): 0.008*"['excellent', '<e>']" + 0.008*"['<s>', 'excellent']" + 0.003*"['classic', '<e>']" + 0.003*"['<s>', '<e>']" + 0.002*"['your', 'foot']" + 0.001*"['favorite', 'shoe']" + 0.001*"['shoe', 'ever']" + 0.001*"['<s>', 'best']" + 0.001*"['ever', '<e>']" + 0.001*"['my', 'favorite']"
2017-12-09 15:40:01,817 : INFO : to

2017-12-09 15:40:08,177 : INFO : topic #8 (0.100): 0.008*"['excellent', '<e>']" + 0.007*"['<s>', 'excellent']" + 0.004*"['classic', '<e>']" + 0.003*"['<s>', '<e>']" + 0.001*"['down', 'a']" + 0.001*"['an', 'and']" + 0.001*"['other', 'review']" + 0.001*"['your', 'foot']" + 0.001*"['favorite', 'shoe']" + 0.001*"['full', 'size']"
2017-12-09 15:40:08,179 : INFO : topic #1 (0.100): 0.007*"['<s>', 'my']" + 0.006*"['daughter', 'love']" + 0.005*"['my', 'daughter']" + 0.004*"['son', 'love']" + 0.004*"['them', '<e>']" + 0.004*"['love', 'these']" + 0.004*"['love', 'them']" + 0.004*"['these', '<e>']" + 0.003*"['my', 'son']" + 0.003*"['<s>', 'excelent']"
2017-12-09 15:40:08,181 : INFO : topic #7 (0.100): 0.007*"['nice', '<e>']" + 0.005*"['<s>', 'nice']" + 0.004*"['<s>', 'very']" + 0.004*"['comfortable', 'shoe']" + 0.003*"['sneaker', '<e>']" + 0.003*"['<s>', 'comfortable']" + 0.003*"['love', 'my']" + 0.002*"['very', 'nice']" + 0.002*"['shoe', '<e>']" + 0.002*"['comfortable', '<e>']"
2017-12-09 15:40:

## Playing around with what the package has to offer

In [207]:
ldamodel.show_topic(topicid=0)

[("['<s>', 'awesome']", 0.0067402918524004478),
 ("['awesome', '<e>']", 0.0050893368415772479),
 ("['<s>', 'classic']", 0.0042127454644020103),
 ("['awesome', 'shoe']", 0.0034254824309236402),
 ("['thanks', '<e>']", 0.0031434088221124457),
 ("['ok', '<e>']", 0.0028883216983020769),
 ("['for', '<e>']", 0.0025097714957464782),
 ("['<s>', 'ok']", 0.0023989436630064758),
 ("['<s>', 'thanks']", 0.0021875422070107759),
 ("['loved', 'them']", 0.001946983630795093)]

In [208]:
ldamodel.print_topics(num_topics=n_topics, num_words=5)

2017-12-09 15:41:05,983 : INFO : topic #0 (0.100): 0.007*"['<s>', 'awesome']" + 0.005*"['awesome', '<e>']" + 0.004*"['<s>', 'classic']" + 0.003*"['awesome', 'shoe']" + 0.003*"['thanks', '<e>']"
2017-12-09 15:41:05,985 : INFO : topic #1 (0.100): 0.007*"['<s>', 'my']" + 0.006*"['daughter', 'love']" + 0.005*"['my', 'daughter']" + 0.004*"['son', 'love']" + 0.004*"['them', '<e>']"
2017-12-09 15:41:05,986 : INFO : topic #2 (0.100): 0.004*"['they', 'are']" + 0.004*"['<s>', 'i']" + 0.004*"['these', 'shoe']" + 0.004*"['a', 'size']" + 0.003*"['fit', 'perfectly']"
2017-12-09 15:41:05,988 : INFO : topic #3 (0.100): 0.006*"['comfortable', '<e>']" + 0.006*"['very', 'comfortable']" + 0.006*"['<s>', 'very']" + 0.002*"['cool', '<e>']" + 0.002*"['my', 'husband']"
2017-12-09 15:41:05,990 : INFO : topic #4 (0.100): 0.014*"['great', '<e>']" + 0.014*"['perfect', '<e>']" + 0.013*"['<s>', 'perfect']" + 0.012*"['it', '<e>']" + 0.012*"['<s>', 'great']"
2017-12-09 15:41:05,992 : INFO : topic #5 (0.100): 0.019*"[

[(0,
  '0.007*"[\'<s>\', \'awesome\']" + 0.005*"[\'awesome\', \'<e>\']" + 0.004*"[\'<s>\', \'classic\']" + 0.003*"[\'awesome\', \'shoe\']" + 0.003*"[\'thanks\', \'<e>\']"'),
 (1,
  '0.007*"[\'<s>\', \'my\']" + 0.006*"[\'daughter\', \'love\']" + 0.005*"[\'my\', \'daughter\']" + 0.004*"[\'son\', \'love\']" + 0.004*"[\'them\', \'<e>\']"'),
 (2,
  '0.004*"[\'they\', \'are\']" + 0.004*"[\'<s>\', \'i\']" + 0.004*"[\'these\', \'shoe\']" + 0.004*"[\'a\', \'size\']" + 0.003*"[\'fit\', \'perfectly\']"'),
 (3,
  '0.006*"[\'comfortable\', \'<e>\']" + 0.006*"[\'very\', \'comfortable\']" + 0.006*"[\'<s>\', \'very\']" + 0.002*"[\'cool\', \'<e>\']" + 0.002*"[\'my\', \'husband\']"'),
 (4,
  '0.014*"[\'great\', \'<e>\']" + 0.014*"[\'perfect\', \'<e>\']" + 0.013*"[\'<s>\', \'perfect\']" + 0.012*"[\'it\', \'<e>\']" + 0.012*"[\'<s>\', \'great\']"'),
 (5,
  '0.019*"[\'<s>\', \'love\']" + 0.016*"[\'them\', \'<e>\']" + 0.015*"[\'love\', \'them\']" + 0.012*"[\'shoe\', \'<e>\']" + 0.012*"[\'great\', \'shoe\']"'

In [209]:
extra_removal_5 = {"nice","awesome","love","loved","great","excellent","like","really","perfect","happy","foot","good","amazing","stan","smith"}

extra_removal_2 = {"bad"}

In [210]:
[i for i in ldamodel.get_document_topics(corpus_tfidf)[:1]]

[[(0, 0.013485140449572764),
  (1, 0.013485140861414774),
  (2, 0.34643795801548283),
  (3, 0.033720581623147318),
  (4, 0.013516125760774355),
  (5, 0.039546862105071183),
  (6, 0.013485141696112363),
  (7, 0.49935255309772575),
  (8, 0.01348535556551876),
  (9, 0.013485140825179799)]]

In [211]:
#[i for i in ldamodel.get_document_topics(corpus_tfidf)[:3]]

In [212]:
topic_assignment = dict()
for doc,all_topics in enumerate(ldamodel.get_document_topics(corpus_tfidf)):
    maxim = 0
    final_doc = None
    final_top = None
    for topic in all_topics:        
        if topic[1]>maxim:
            final_doc,final_top, maxim = doc,topic[0],topic[1]
    topic_assignment[doc] = [final_top,maxim,doc_complete[doc]]
    
    
topic_df_0 = pd.DataFrame.from_dict(topic_assignment,orient="index")
topic_df_0.columns = ["Topic","prob","text"]

topic_list = []
for topic in range(10):
    x=[]
    for i in ldamodel.show_topic(topicid=topic)[0:2]:
        x.append(i[0])
        
    topic_list.append([topic, str(x)])
    
topic_names= pd.DataFrame(topic_list)
topic_names.columns = ["Topic","Topic_header"]

topic_df = topic_df_0.merge(topic_names,on="Topic")

#topic_assignment

In [214]:
topic_df_0.head()

,Topic,prob,text
0,7,0.4993,I wear a size 8 in women's so I ordered a size...
1,2,0.7589,My daughter is in love with these. the first p...
2,8,0.5590,Just love these shoes! And cheaper than anywhe...
3,3,0.6094,Met my expectations perfectly! They were sold ...
4,2,0.4424,Best sneakers I have ever bought from a comfor...


In [217]:
topic_df.Topic.value_counts()

2    4931
4    4108
7    2490
5    2073
8    1766
9    1512
3    1379
6    1322
0    1270
1    1247
Name: Topic, dtype: int64

In [218]:
topic_names

,Topic,Topic_header
0,0,"[""['<s>', 'awesome']"", ""['awesome', '<e>']""]"
1,1,"[""['<s>', 'my']"", ""['daughter', 'love']""]"
2,2,"[""['they', 'are']"", ""['<s>', 'i']""]"
3,3,"[""['comfortable', '<e>']"", ""['very', 'comforta..."
4,4,"[""['great', '<e>']"", ""['perfect', '<e>']""]"
5,5,"[""['<s>', 'love']"", ""['them', '<e>']""]"
6,6,"[""['product', '<e>']"", ""['great', 'product']""]"
7,7,"[""['nice', '<e>']"", ""['<s>', 'nice']""]"
8,8,"[""['excellent', '<e>']"", ""['<s>', 'excellent']""]"
9,9,"[""['very', 'good']"", ""['<s>', 'very']""]"


In [219]:
topic_df[topic_df.Topic==1].sort_values(by="prob",ascending=False).head(10)

,Topic,prob,text,Topic_header
11581,1,0.8904,"Soy unos zapatos muy cómodos, ademas son muy f...","[""['<s>', 'my']"", ""['daughter', 'love']""]"
11387,1,0.8843,E' la prima volta che ho acquistato qualcosa s...,"[""['<s>', 'my']"", ""['daughter', 'love']""]"
10591,1,0.8593,en ouvrant la boite je les est vues et pris da...,"[""['<s>', 'my']"", ""['daughter', 'love']""]"
11322,1,0.8388,"zapato con suela color crema, todos los demas ...","[""['<s>', 'my']"", ""['daughter', 'love']""]"
11329,1,0.8162,"Excelente, buen zapato, la calidad es compatib...","[""['<s>', 'my']"", ""['daughter', 'love']""]"
11767,1,0.7664,"Great quality,great price,fast shipping,will s...","[""['<s>', 'my']"", ""['daughter', 'love']""]"
11265,1,0.7649,Il est arrivé 3 jour à l'avance j'adore,"[""['<s>', 'my']"", ""['daughter', 'love']""]"
10597,1,0.7649,Es posible cambiar por un talle menos?,"[""['<s>', 'my']"", ""['daughter', 'love']""]"
11774,1,0.7449,Mu son loves these brand of shoes!!,"[""['<s>', 'my']"", ""['daughter', 'love']""]"
10699,1,0.7431,very bright color! My son loves them!,"[""['<s>', 'my']"", ""['daughter', 'love']""]"


In [220]:
topic_df.text.loc[1495]

'Nice'

In [221]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus_tfidf, dictionary)

/Users/Kevin/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix



PreparedData(topic_coordinates=         Freq  cluster  topics       x       y
topic                                         
2     24.5177        1       1 -0.2241 -0.0797
4     12.6263        1       2 -0.0783  0.1642
7     11.5892        1       3  0.0260  0.0041
8      9.6485        1       4  0.0657 -0.0318
5      8.3589        1       5  0.0074  0.0159
6      7.3614        1       6  0.0535 -0.0156
3      7.2826        1       7  0.0469 -0.0193
0      6.6222        1       8  0.0457 -0.0262
1      6.0261        1       9  0.0241 -0.0213
9      5.9672        1      10  0.0331  0.0098, topic_info=      Category     Freq                      Term    Total  loglift  logprob
term                                                                        
364    Default 232.0000           ['<s>', 'love'] 232.0000  30.0000  30.0000
525    Default 156.0000          ['great', '<e>'] 156.0000  29.0000  29.0000
561    Default 141.0000        ['<s>', 'perfect'] 141.0000  28.0000  28.0000
393    Default 122.0000             ['it', '<e>'] 122.0000  27.0000  27.0000
164    Default 192.0000          ['love', 'them'] 192.0000  26.0000  26.0000
45     Default 147.0000        ['perfect', '<e>'] 147.0000  25.0000  25.0000
255    Default 195.0000           ['them', '<e>'] 195.0000  24.0000  24.0000
392    Default 107.0000            ['love', 'it'] 107.0000  23.0000  23.0000
2053   Default 123.0000           ['good', '<e>'] 123.0000  22.0000  22.0000
294    Default 238.0000          ['<s>', 'great'] 238.0000  21.0000  21.0000
835    Default 120.0000         ['great', 'shoe'] 120.0000  20.0000  20.0000
2052   Default 118.0000           ['<s>', 'good'] 118.0000  19.0000  19.0000
332    Default 179.0000           ['shoe', '<e>'] 179.0000  18.0000  18.0000
1636   Default  66.0000           ['nice', '<e>']  66.0000  17.0000  17.0000
1865   Default 128.0000           ['<s>', 'very'] 128.0000  16.0000  16.0000
1284   Default  59.0000      ['excellent', '<e>']  59.0000  15.0000  15.0000
651    Default  52.0000          ['very', 'good']  52.0000  14.0000  14.0000
46     Default 106.0000             ['<s>', 'my'] 106.0000  13.0000  13.0000
1184   Default  76.0000      ['<s>', 'excellent']  76.0000  12.0000  12.0000
1635   Default  83.0000           ['<s>', 'nice']  83.0000  11.0000  11.0000
417    Default  62.0000         ['a', 'expected']  62.0000  10.0000  10.0000
0      Default 144.0000              ['<s>', 'i'] 144.0000   9.0000   9.0000
827    Default 119.0000    ['comfortable', '<e>'] 119.0000   8.0000   8.0000
1237   Default  50.0000       ['expected', '<e>']  50.0000   7.0000   7.0000
88     Default 108.0000         ['love', 'these'] 108.0000   6.0000   6.0000
1707   Default  36.0000          ['nice', 'shoe']  36.0000   5.0000   5.0000
47     Default  54.0000        ['my', 'daughter']  54.0000   4.0000   4.0000
928    Default 113.0000   ['very', 'comfortable'] 113.0000   3.0000   3.0000
1795   Default  45.0000        ['<s>', 'awesome']  45.0000   2.0000   2.0000
943    Default  39.0000      ['daughter', 'love']  39.0000   1.0000   1.0000
...        ...      ...                       ...      ...      ...      ...
277    Topic10  16.9958            ['you', '<e>']  19.2183   2.6960  -5.6290
4597   Topic10   3.1025           ['knew', 'the']   3.5119   2.6949  -7.3298
11078  Topic10   3.0944             ['last', 'a']   3.5044   2.6945  -7.3324
16808  Topic10   3.0741         ['sandal', 'are']   3.4832   2.6940  -7.3390
8992   Topic10   3.0235  ['great', 'comfortable']   3.4317   2.6923  -7.3555
2444   Topic10  11.5765       ['good', 'product']  13.1717   2.6898  -6.0130
4981   Topic10   2.9201         ['great', 'they']   3.3303   2.6874  -7.3904
651    Topic10  45.6629          ['very', 'good']  52.1743   2.6856  -4.6407
2671   Topic10   6.8898        ['quickly', 'and']   7.8732   2.6855  -6.5319
5647   Topic10   2.8271        ['ordered', 'and']   3.2361   2.6838  -7.4227
276    Topic10  18.7777          ['thank', 'you']  25.6051   2.5088  -5.5293
5919 

In [230]:
from gensim import corpora

documents = ["Human machine interface for lab abc computer applications",
          "A survey of user opinion of computer system response time",
          "The EPS user interface management system",
          "System and human system engineering testing of EPS",
          "Relation of user perceived response time to error measurement",
          "The generation of random binary unordered trees",
          "The intersection graph of paths in trees",
          "Graph minors IV Widths of trees and well quasi ordering",
          "Graph minors A survey"]

In [241]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
      for document in documents]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
 for token in text:
     frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
      for text in texts]

from pprint import pprint  # pretty-printer
pprint(texts)


[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [233]:
dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/deerwester.dict')  # store the dictionary, for future reference
print(dictionary)
#Dictionary(12 unique tokens)

print(dictionary.token2id)
{'minors': 11, 'graph': 10, 'system': 5, 'trees': 9, 'eps': 8, 'computer': 0,
'survey': 4, 'user': 7, 'human': 1, 'time': 6, 'interface': 2, 'response': 3}

2017-12-03 11:12:52,757 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-12-03 11:12:52,758 : INFO : built Dictionary(12 unique tokens: ['human', 'interface', 'computer', 'survey', 'user']...) from 9 documents (total 29 corpus positions)
2017-12-03 11:12:52,759 : INFO : saving Dictionary object under /tmp/deerwester.dict, separately None
2017-12-03 11:12:52,761 : INFO : saved /tmp/deerwester.dict


Dictionary(12 unique tokens: ['human', 'interface', 'computer', 'survey', 'user']...)
{'human': 0, 'interface': 1, 'computer': 2, 'survey': 3, 'user': 4, 'system': 5, 'response': 6, 'time': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


{'computer': 0,
 'eps': 8,
 'graph': 10,
 'human': 1,
 'interface': 2,
 'minors': 11,
 'response': 3,
 'survey': 4,
 'system': 5,
 'time': 6,
 'trees': 9,
 'user': 7}

In [243]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)  # the word "interaction" does not appear in the dictionary and is ignored

[(0, 1), (2, 1)]


In [244]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus)  # store to disk, for later use
print(corpus)


2017-12-03 11:16:39,752 : INFO : storing corpus in Matrix Market format to /tmp/deerwester.mm
2017-12-03 11:16:39,754 : INFO : saving sparse matrix to /tmp/deerwester.mm
2017-12-03 11:16:39,755 : INFO : PROGRESS: saving document #0
2017-12-03 11:16:39,756 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2017-12-03 11:16:39,757 : INFO : saving MmCorpus index to /tmp/deerwester.mm.index


[[(0, 1), (1, 1), (2, 1)], [(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(1, 1), (4, 1), (5, 1), (8, 1)], [(0, 1), (5, 2), (8, 1)], [(4, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(3, 1), (10, 1), (11, 1)]]


In [236]:

if (os.path.exists("/tmp/deerwester.dict")):
    dictionary = corpora.Dictionary.load('/tmp/deerwester.dict')
    corpus = corpora.MmCorpus('/tmp/deerwester.mm')
    print("Used files generated from first tutorial")
else:
    print("Please run first tutorial to generate data set")

2017-12-03 11:13:22,805 : INFO : loading Dictionary object from /tmp/deerwester.dict
2017-12-03 11:13:22,807 : INFO : loaded /tmp/deerwester.dict
2017-12-03 11:13:22,808 : INFO : loaded corpus index from /tmp/deerwester.mm.index
2017-12-03 11:13:22,809 : INFO : initializing corpus reader from /tmp/deerwester.mm
2017-12-03 11:13:22,811 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


Used files generated from first tutorial


In [245]:
tfidf = models.TfidfModel(corpus)

2017-12-03 11:17:00,732 : INFO : collecting document frequencies
2017-12-03 11:17:00,733 : INFO : PROGRESS: processing document #0
2017-12-03 11:17:00,735 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


In [246]:
corpus

[[(0, 1), (1, 1), (2, 1)],
 [(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(1, 1), (4, 1), (5, 1), (8, 1)],
 [(0, 1), (5, 2), (8, 1)],
 [(4, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(3, 1), (10, 1), (11, 1)]]

In [249]:
tfidf

In [251]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow]) # step 2 -- use the model to transform vectors
#[(0, 0.70710678), (1, 0.70710678)]

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [254]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print (doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(2, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.3244870206138555), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.44424552527467476)]
[(1, 0.5710059809418182), (4, 0.4170757362022777), (5, 0.4170757362022777), (8, 0.5710059809418182)]
[(0, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(4, 0.45889394536615247), (6, 0.6282580468670046), (7, 0.6282580468670046)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(3, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


# Ignore Below

In [291]:
reviews = pd.read_csv("reviews.csv")

In [292]:
twitter = pd.read_csv("twitter.csv",header=5)

In [293]:
twitter.columns

Index(['Source', 'Host', 'Link', 'Time(ET)', 'Authority', 'Age', 'Gender',
       'Country', 'Location', 'Tags', 'Tags.1', 'Tags.2', 'Tags.3', 'Tags.4',
       'Tags.5', 'Tags.6', 'Star', 'Assigned', 'Sentiment', 'Alexa Rank',
       'Alexa Reach', 'Title', 'Snippet', 'Content', 'Unique ID', 'Language',
       'Followers', 'Following', 'Summary', 'Viewed', 'Commented',
       'Favourited', 'Rated', 'ImageUrl'],
      dtype='object')

In [294]:
reviews[pd.isnull(reviews.Text)==False].shape

(17766, 18)

In [295]:
reviews.head()

,Month/Year,Submission Day,# of Reviews,ASIN,Category,Product,YTD Revenue,YTD Units,LTD Star Rating,LTD Total # of Reviews,Replen Code,Monthly Star Rating,Verified?,Review Title,Text,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,Jan-17,03-JAN-17,1.0000,B00SG7M6JW,Latch Bottles,"Latch BPA-Free Baby Bottle, 4 Ounce, 3 Pack",nan,nan,4.2800,38.0000,Planned Replenishment (PR),5,Y,NaN,I bought these as a supplement to our Tommee T...,NaN,NaN,NaN
1,Jan-17,02-JAN-17,1.0000,B006SFUDQM,Bath,"Bubble Spout Guard, Blue",nan,nan,3.0300,192.0000,Planned Replenishment (PR),3,Y,NaN,This is a hard spout guard so I don't imagine ...,NaN,NaN,NaN
2,Jan-17,02-JAN-17,1.0000,B00J40DY0A,Latch Bottles,"LATCH Transition Cup, 4 Ounce",nan,nan,4.0500,592.0000,Planned Replenishment (PR),5,Y,NaN,Perfect,NaN,NaN,NaN
3,Jan-17,01-JAN-17,1.0000,B00PHWMKVS,Bath,"Wind Up Swimming Penguin Bath Toy, Pink",nan,nan,4.1500,529.0000,Planned Replenishment (PR),5,Y,NaN,This is adorable! My daughter loves it. The wa...,NaN,NaN,NaN
4,Jan-17,02-JAN-17,1.0000,B009R27JVU,Diapering,"Arm & Hammer Diaper Pail Snap, Seal and Toss R...",nan,nan,4.0900,397.0000,Planned Replenishment (PR),5,N,NaN,Work well with my arm and hammer pail. The sce...,NaN,NaN,NaN


In [296]:
import datetime

In [297]:
datetime.datetime.strptime("03-JAN-17", '%d-%b-%y')

datetime.datetime(2017, 1, 3, 0, 0)

In [298]:
def to_date(x):
    if pd.isnull(x) == False:
        return datetime.datetime.strptime(x, '%d-%b-%y')

In [299]:
reviews["date"] = reviews["Submission Day"].apply(to_date)

In [300]:
reviews["rating"] = reviews['Monthly Star Rating']

In [301]:
reviews[['date','Product','Category','rating','Text']].head()

,date,Product,Category,rating,Text
0,2017-01-03,"Latch BPA-Free Baby Bottle, 4 Ounce, 3 Pack",Latch Bottles,5,I bought these as a supplement to our Tommee T...
1,2017-01-02,"Bubble Spout Guard, Blue",Bath,3,This is a hard spout guard so I don't imagine ...
2,2017-01-02,"LATCH Transition Cup, 4 Ounce",Latch Bottles,5,Perfect
3,2017-01-01,"Wind Up Swimming Penguin Bath Toy, Pink",Bath,5,This is adorable! My daughter loves it. The wa...
4,2017-01-02,"Arm & Hammer Diaper Pail Snap, Seal and Toss R...",Diapering,5,Work well with my arm and hammer pail. The sce...


In [302]:
import numpy as np
import re

## Later take a look at similarities by specific star rating
* Decided to look at full corpus for now

In [303]:
one_star_text = ''

for text in reviews.Text[reviews.rating == 1]:
    one_star_text += '<s>' + re.sub(r'[^\w\s]',' ',re.sub("'",'',text.lower()))
    how_many += 1
    
    

In [304]:
re.sub(' +',' ','The     quick brown    fox')

'The quick brown fox'

In [305]:
twitter_text = ''

for text in twitter.Content:
    twitter_text += ' <s> ' + re.sub(r'[^\w\s]',' ',re.sub("'",'',str(text).lower()))

In [306]:
twitter_text = re.sub(' +',' ',twitter_text)

In [307]:
twitter_text[:300]

' <s> join me mon nov 20 at humbercinemas for a movies4mommies screening of murder on the orient express doors open at 12 30pm screening is at 1pm enter for a chance to win prizes from munchkin_inc dkcanada plus get a clubcard so your 7th movie is free moviesformommies https t co zebaqewm81\n <s> plea'

In [308]:
amazon_text = ''
amazon_text_list = []

#how_many =0

for text in reviews.Text:
    amazon_text += ' <s> ' + re.sub(r'[^\w\s]',' ',re.sub("'",'',str(text).lower()))+ ' <\s> '
    #how_many+=1
    
for text in reviews.Text:
    amazon_text_list.append(' <s> ' + re.sub(r'[^\w\s]',' ',re.sub("'",'',str(text).lower()))+ ' <\s> ')   


In [309]:
amazon_text = re.sub(' +',' ',all_text)
amazon_text[:500]

' <s> i bought these as a supplement to our tommee tippee closer to nature bottles and i think i like these more unlike tommee tippee where the venting happens through the nipple the venting happens on the bottom so theres no risk of milk dripping out and leaking and making a mess when you breastfeed every drop counts so thats a huge plus ive tested these quite vigorously and i cant get the vents to leak at all even when submerged so thats a win the slow flow nipples that come with i <s> this is '

In [310]:
amazon_words = amazon_text.split(" ")

In [311]:
from collections import defaultdict

In [312]:
word_count = defaultdict(int)



for word in amazon_words:
    word_count[word]+=1
    

    

In [313]:
wc_df =pd.DataFrame.from_dict(word_count,orient="index")

In [314]:
#wc_df['word'] = wc_df.index

In [315]:
wc_df.reset_index(inplace=True)
wc_df.columns = ["word","count"]
wc_df['index_1'] = wc_df.index

In [316]:
wc_df.sort_values(by=["count","word"],ascending=False,inplace=True)

In [317]:
wc_df.tail()

,word,count,index_1
6653,0tool,1,6653
10793,07,1,10793
12963,02,1,12963
5690,01,1,5690
7619,00pm,1,7619


In [318]:
word_id = {}

for i,row in wc_df.iterrows():
    word_id[row.word]=i

In [319]:
word_id['to']

8

In [320]:
def cooccurrence_matrix(token_ids, V, K=2):
    # We'll use this as an "accumulator" matrix
    C = scipy.sparse.csc_matrix((V,V), dtype=np.float32)

    for k in range(1, K+1):
        print (u"Counting pairs (i, i \u00B1 %d) ..." % k)
        i = token_ids[:-k]  # current word
        j = token_ids[k:]   # k words ahead
        data = (np.ones_like(i), (i,j))  # values, indices
        Ck_plus = scipy.sparse.coo_matrix(data, shape=C.shape, dtype=np.float32)
        Ck_plus = scipy.sparse.csc_matrix(Ck_plus)
        Ck_minus = Ck_plus.T  # Consider k words behind
        C += Ck_plus + Ck_minus

    print ("Co-occurrence matrix: %d words x %d words" % (C.shape))
    print ("  %.02g nonzero elements" % (C.nnz))
    return C

In [321]:
amazon_words[:10]

['', '<s>', 'i', 'bought', 'these', 'as', 'a', 'supplement', 'to', 'our']

In [322]:
# Show co-occurrence on a toy corpus


amazon_text_list

#words


amazon_words

# ?

# sentence_to_ids adds "<s>" and "</s>"

#gives id of words



amazon_words_ids = [word_id[s] for s in amazon_words]

# Here's the important part


amazon_C = cooccurrence_matrix(amazon_words_ids, len(amazon_words), K = 1)

#ORdered from most common to least common than reverse alphabetical

#pretty_print_matrix(amazon_C.toarray(), rows=wc_df.word, 
#                          cols=wc_df.word, dtype=int, highlight="> 0")

Counting pairs (i, i ± 1) ...
Co-occurrence matrix: 663684 words x 663684 words
  3.1e+05 nonzero elements


In [323]:
def PPMI(C):
    """Tranform a counts matrix to PPMI.
    
    Args:
      C: scipy.sparse.csc_matrix of counts C_ij
    
    Returns:
      (scipy.sparse.csc_matrix) PPMI(C) as defined above
    """
    Z = float(C.sum())  # total counts
    # sum each column (along rows)
    Zc = np.array(C.sum(axis=0), dtype=np.float64).flatten()
    # sum each row (along columns)
    Zr = np.array(C.sum(axis=1), dtype=np.float64).flatten()
    
    # Get indices of relevant elements
    ii, jj = C.nonzero()  # row, column indices
    Cij = np.array(C[ii,jj], dtype=np.float64).flatten()
    
    ##
    # PMI equation
    pmi = np.log(Cij * Z / (Zr[ii] * Zc[jj]))
    ##
    # Truncate to positive only
    ppmi = np.maximum(0, pmi)  # take positive only
    
    # Re-format as sparse matrix
    ret = scipy.sparse.csc_matrix((ppmi, (ii,jj)), shape=C.shape,
                                  dtype=np.float64)
    ret.eliminate_zeros()  # remove zeros
    return ret

In [324]:
from sklearn.decomposition import TruncatedSVD
def SVD(X, d=100):
    """Returns word vectors from SVD.
    
    Args:
      X: m x n matrix
      d: word vector dimension
      
    Returns:
      Wv : m x d matrix, each row is a word vector.
    """
    transformer = TruncatedSVD(n_components=d, random_state=1)
    Wv = transformer.fit_transform(X)
    # Normalize to unit length
    Wv = Wv / np.linalg.norm(Wv, axis=1).reshape([-1,1])
    return Wv, transformer.explained_variance_

In [325]:
K = 1
d = 100
t0 = time.time()
C = cooccurrence_matrix(amazon_words_ids, len(amazon_words), K=K)

C_ppmi = PPMI(C)

Wv, _ = SVD(C_ppmi, d=d)


Counting pairs (i, i ± 1) ...
Co-occurrence matrix: 663684 words x 663684 words
  3.1e+05 nonzero elements


/Users/Kevin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning:

invalid value encountered in true_divide



In [286]:
n = 1000
data = [go.Scatter(x=Wv[:n,0], y=Wv[:n,1], text=amazon_words[:n],
                   mode='markers', textposition='bottom', hoverinfo='text')]
fig = go.Figure(data=data, layout=go.Layout(title="Word Embeddings", hovermode='closest'))
plotly.iplot(fig)